# 1. Scale Model

In [1]:
import os
import numpy as np
import opensim as osim

## Inputs

In [2]:
# root directory of the opensim project
root = '/home/romain/Downloads/opensim/'

# participant
participant = 'SylGaud'

# model file name
model_name = 'model.osim'

# static trial name
static_trial_name = 'SylGaud0.trc'

# scaling configuration
scaling_conf_name = 'scaling.xml'

In [3]:
model = {
    'name': model_name,
    'path': os.path.join(root, 'metadata', 'generic', model_name),
    'output': os.path.join(root, 'metadata', participant, 'scaled-{}'.format(model_name))
}

static_trial = {
    'name': static_trial_name,
    'path': os.path.join(root, 'outputs', participant, '0_trc', static_trial_name)
}

scaling_conf = {
    'name': scaling_conf_name,
    'path': os.path.join(root, 'metadata', 'generic', scaling_conf_name),
    'output': os.path.join(root, 'metadata', participant, 'scaled-{}'.format(scaling_conf_name))
}

## Scaling

In [4]:
# setup scale tool
scale_tool = osim.ScaleTool(scaling_conf['path'])
scale_tool.setName('scaled-{}'.format(model['name'].replace('.osim', '')))
scale_tool.getGenericModelMaker().setModelFileName(model['path'])

# get initial and final time
marker_data = osim.MarkerData(static_trial['path'])
initial_time = marker_data.getStartFrameTime()
final_time = marker_data.getLastFrameTime()
range_time = osim.ArrayDouble()
range_time.set(0, initial_time)
range_time.set(1, final_time)
# NOT WORKING HERE

# model scaler
model_scaler = scale_tool.getModelScaler()
model_scaler.setMarkerFileName(static_trial['path'])
model_scaler.setTimeRange(range_time)
model_scaler.setOutputModelFileName(model['output'])
model_scaler.setOutputScaleFileName(scaling_conf['output'])

# run model scaler
gen_model = osim.Model(model['path'])
#state = gen_model.initSystem()
model_scaler.processModel(gen_model)

# IK
marker_placer = scale_tool.getMarkerPlacer()
marker_placer.setStaticPoseFileName(static_trial['path'])
marker_placer.setTimeRange(range_time)
marker_placer.setOutputModelFileName(model['output'].replace('model', 'model-MKR'))
scaled_model = osim.Model(model['output'])
#state = scaled_model.initSystem()
marker_placer.processModel(scaled_model)

scale_tool.printToXML(scaling_conf['output'])

True